In [1]:
import numpy as np

with open('reviews.txt','r') as f:
    reviews = f.read()
with open('labels.txt','r') as f:
    labels = f.read()

In [2]:
print(reviews[:1000])
print()
print(labels[:30])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [3]:
from string import punctuation

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')
all_text = ''.join(reviews_split)

words = all_text.split()

In [4]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [5]:
from collections import Counter

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}


reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [6]:
print('Unique words: ', len((vocab_to_int)))

print()

print('Tokenized review: \n', reviews_ints[:2])

Unique words:  53766

Tokenized review: 
 [[14010, 319, 6, 3, 1118, 202, 8, 2214, 32, 1, 168, 57, 15, 49, 78, 5681, 44, 420, 114, 140, 15, 4026, 61, 147, 9, 1, 4783, 5682, 481, 70, 5, 258, 12, 14010, 319, 13, 2263, 6, 73, 2507, 5, 593, 74, 6, 4026, 1, 17942, 5, 1889, 9261, 1, 5969, 1557, 36, 52, 65, 207, 145, 67, 1240, 4026, 19990, 1, 26843, 4, 1, 225, 881, 31, 3159, 70, 4, 1, 6119, 10, 703, 2, 67, 1557, 55, 10, 212, 1, 353, 9, 64, 3, 1390, 3633, 756, 5, 3542, 177, 1, 420, 10, 1187, 16357, 32, 319, 3, 347, 334, 3083, 10, 143, 129, 5, 7036, 30, 4, 130, 4026, 1390, 2452, 5, 14010, 319, 10, 525, 12, 110, 1403, 4, 61, 589, 102, 12, 14010, 319, 6, 234, 4164, 48, 3, 2453, 12, 8, 230, 21], [62, 4, 3, 125, 36, 47, 9262, 1252, 16, 3, 3429, 499, 45, 17, 3, 637, 132, 12, 6, 3, 1271, 468, 4, 1729, 202, 3, 10552, 8585, 292, 6, 645, 89, 35, 2591, 1085, 3084, 34, 1, 898, 33695, 4, 8, 13, 5126, 487, 8, 2550, 1729, 1, 225, 57, 17, 60, 747, 1399, 887, 227, 8, 42, 98, 124, 1536, 58, 146, 37, 1, 963, 139,

In [7]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [8]:
len(encoded_labels)

25000

In [9]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero length reviews: {}".format(review_lens[0]))
print("Maximum length: {}".format(max(review_lens)))

Zero length reviews: 0
Maximum length: 2514


In [10]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print("Number of reviews after removing outliers: ",len(reviews_ints))


Number of reviews before removing outliers:  12110
Number of reviews after removing outliers:  12110


In [11]:
features = np.zeros((len(reviews_ints), 100), dtype=int)
for i , row in enumerate(reviews_ints):
        x = np.array(row)[:50]
        features[i, -len(x):] = x

In [16]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i , row in enumerate(reviews_ints):
        x = np.array(row)[:seq_length]
        features[i, :len(x)] = x
    return features

In [18]:
seq_length = 100

features = pad_features(reviews_ints[:12110], seq_length=seq_length)

assert len(features) == len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0]) == seq_length, "Each feature row should contain seq_length values."

print(features[:2,:10])

[[14010   319     6     3  1118   202     8  2214    32     1]
 [   62     4     3   125    36    47  9262  1252    16     3]]


In [20]:
split_frac = 0.8

split_idx = int(len(features)*split_frac)

train_x, remaining_x = features[:split_idx] , features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)

val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print("\t\t\t Features Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
     "\nValidation set: \t{}".format(val_x.shape),
     "\nTest set: \t\t{}".format(test_x.shape))

			 Features Shapes:
Train set: 		(9688, 100) 
Validation set: 	(1211, 100) 
Test set: 		(1211, 100)


In [21]:
import torch 
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))

batch_size = 50

train_loader  = DataLoader(train_data, shuffle=True , batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


In [22]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size())
print('Sample input: \n',sample_x)
print()
print('Sample label size: ',sample_y.size())
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 100])
Sample input: 
 tensor([[    1,  6804,  1438,  ...,     1,  3842, 29695],
        [   16,   425,     4,  ...,    85,    53,     4],
        [ 1167,    37,     3,  ...,    21,   122,    25],
        ...,
        [   37,     1,   338,  ...,     7,     7,  3544],
        [   11,    14,     1,  ...,    41,     6,    60],
        [  100,   306,    63,  ...,  1700,  8981,     3]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0], dtype=torch.int32)


In [25]:
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [27]:
import torch.nn as nn
class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
        
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out , hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out[:,-1,:]
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        sig_out = self.sig(out)
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                 weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden
    

In [28]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentRNN(
  (embedding): Embedding(53767, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)
